# Drzewa sufiksów

1. Przyjmij następujący zbiór danych wejściowych:
    1. bbb\$
    2. aabbabd
    3. ababcd
    4. abaababaabaabaabab\$
    5. losowy tekst o długości 100 znaków,
    6. załączony plik.\
Upewnij się, że każdy łańcuch na końcu posiada unikalny znak (marker), a jeśli go nie ma, to dodaj ten znak.
2. Zaimplementuj algorytm konstruujący strukturę trie, która przechowuje wszystkie sufiksy łańcucha danego na wejściu.
3. Zaimplementuj algorytm konstruujący drzewo sufiksów.
4. Upewnij się, że powstałe struktury danych są poprawne. Możesz np. sprawdzić, czy struktura zawiera jakiś ciąg znaków i porównać wyniki z algorytmem wyszukiwania wzorców.
5. Porównaj szybkość działania algorytmów konstruujących struktury danych dla danych z p. 1 w następujących wariantach:
    1. Trie - w wariancie, w którym kolejne sufiksy dodawane są przez przeszukiwanie głowy od korzenia drzewa (1p.),
    2. Trie - w wariancie, w którym kolejne sufiksy dodawane są poprzez dodanie kolejnej litery tekstu (1p.),
    3. Drzewo sufiksów - algorytm Ukkonena (3p).\
Dla załączonego tekstu czas wariantów 1 i 2 może być nieakceptowalnie długi - w tej sytuacji pomiń wyniki pomiarów dla tego tekstu.

6. Oczekiwany wynik ćwiczenia to kod źródłowy oraz raport w formie PDF.

In [1]:
from string import printable
from random import choice
from timeit import default_timer as timer
import pandas as pd

## ad 1. Przygotowanie danych wejściowych

In [2]:
dataA='bbb$' #$ - marker
dataB='aabbabd' #d - marker
dataC='ababcd' #d - marker
dataD='abaababaabaabaabab$' #$ - marker

In [3]:
dataE=''.join(choice(printable[1:]) for i in range(99))
dataE+=printable[0] #printable[0] ('0') - marker

In [4]:
with open("1997_714_head.txt") as file:
    dataF = file.read()
dataF += "$" #$ - marker

## ad 2. Suffix trie

### Bez suffix linków

In [5]:
class StaticTrieNode: 
    def __init__(self, parent):
        self.parent = parent
        self.children = dict()
    
    def suffixes(self):
        res=[]
        for child in self.children.keys():
            sts=self.children[child].__str__()
            for s in sts:
                res.append(child+s)
                
        if len(res)==0: return [""]
        return res
        
class StaticTrie:
    def __init__(self, text):
        start=timer()
        self.root = StaticTrieNode(None)
        for i in range(len(text)):
            suffix = text[i:]
            head, index = self.find(suffix)
            self.graft(head, suffix[index:])
        end=timer()
        self.time=end-start
    
    def __str__(self):
        return str(self.root.suffixes())

    def find(self, text):
        current_node = self.root
        idx = 0
        while idx < len(text) and text[idx] in current_node.children:
            current_node = current_node.children[text[idx]]
            idx += 1
        return current_node, idx

    def graft(self, node, text):
        for c in text:
            new_node = StaticTrieNode(node)
            node.children[c] = new_node
            node = new_node

    def pattern_search(self, pattern):
        if len(pattern) == 0:
            return True
        node, index = self.find(pattern)
        return index == len(pattern)

### Z suffix linkami

In [6]:
class DynamicTrieNode: 
    def __init__(self, parent, link=None):
        self.parent = parent
        self.children = dict()
        self.link=link
        
    def suffixes(self):
        res=[]
        for child in self.children.keys():
            sts=self.children[child].suffixes()
            for s in sts:
                res.append(child+s)
                
        if len(res)==0: return [""]
        return res
    
    def add_child(self, c):
        child = DynamicTrieNode(self)
        self.children[c] = child
        return child

    def add_link(self, node):
        self.link = node

        
class DynamicTrie: 
    def __init__(self, text):
        start=timer()
        self.root = DynamicTrieNode(None)
        
        deepest=self.root
        
        for i in range(len(text)):
            c=text[i]
            node=deepest
            deepest=None
            prev=None
            
            while(node and (c not in node.children)):
                child=node.add_child(c)
                node.children[c]=child
                #first iteration
                if (not deepest):
                    deepest=child
                #rest of iterations
                if (prev):
                    prev.add_link(child)
                if node == self.root:
                    child.add_link(self.root)
                prev=child
                node=node.link
            if(node):
                prev.link=node.children[c]
        end=timer()
        self.time=end-start
                
    def __str__(self):
        return str(self.root.suffixes())

    
    def pattern_search(self, pattern):
        node = self.root

        for c in pattern:
            if c not in node.children:
                return False

            node = node.children[c]

        return True

## ad 3. Suffix tree

In [7]:
#:((

## ad 4. Poprawność struktur

In [8]:
strieA=StaticTrie(dataA)
dtrieA=DynamicTrie(dataA)

strieB=StaticTrie(dataB)
dtrieB=DynamicTrie(dataB)

strieC=StaticTrie(dataC)
dtrieC=DynamicTrie(dataC)

strieD=StaticTrie(dataD)
dtrieD=DynamicTrie(dataD)

strieE=StaticTrie(dataE)
dtrieE=DynamicTrie(dataE)

In [9]:
strieF=StaticTrie(dataF)

In [10]:
dtrieF=DynamicTrie(dataF)

In [11]:
print("test StaticTrie A:", end="\t")
print("ok") if strieA.pattern_search("bb") and not strieA.pattern_search("abk") else print("not ok!!!")

print("test DynamicTrie A:", end="\t")
print("ok") if dtrieA.pattern_search("bbb") and not dtrieA.pattern_search("abk") else print("not ok!!!")

test StaticTrie A:	ok
test DynamicTrie A:	ok


In [12]:
print("test StaticTrie B:", end="\t")
print("ok") if strieB.pattern_search("babd") and not strieB.pattern_search("babdb") else print("not ok!!!")

print("test DynamicTrie B:", end="\t")
print("ok") if dtrieB.pattern_search("babd") and not dtrieB.pattern_search("babdb") else print("not ok!!!")

test StaticTrie B:	ok
test DynamicTrie B:	ok


In [13]:
print("test StaticTrie C:", end="\t")
print("ok") if strieC.pattern_search("abc") and not strieC.pattern_search("abbbc") else print("not ok!!!")

print("test DynamicTrie C:", end="\t")
print("ok") if dtrieC.pattern_search("abc") and not dtrieC.pattern_search("abbc") else print("not ok!!!")

test StaticTrie C:	ok
test DynamicTrie C:	ok


In [14]:
print("test StaticTrie D:", end="\t")
print("ok") if strieD.pattern_search("baababa") and not strieD.pattern_search("baababaaaaa") else print("not ok!!!")

print("test DynamicTrie D:", end="\t")
print("ok") if dtrieD.pattern_search("baababa") and not dtrieD.pattern_search("baababaaaaa") else print("not ok!!!")

test StaticTrie D:	ok
test DynamicTrie D:	ok


In [15]:
print("test StaticTrie E:", end="\t")
print("ok") if strieE.pattern_search(dataE[3:50]) and not strieD.pattern_search("00") else print("not ok!!!")

print("test DynamicTrie E:", end="\t")
print("ok") if dtrieE.pattern_search(dataE[3:50]) and not dtrieD.pattern_search("00") else print("not ok!!!")

test StaticTrie E:	ok
test DynamicTrie E:	ok


In [16]:
testok="163, Nr 90, poz. 419, Nr 113, poz."
testnotok="poz. 419, Nr 113ala ma kota"

print("test StaticTrie F:", end="\t")
print("ok") if strieF.pattern_search(testok) and not strieD.pattern_search(testnotok) else print("not ok!!!")

print("test DynamicTrie F:", end="\t")
print("ok") if dtrieF.pattern_search(testok) and not dtrieD.pattern_search(testnotok) else print("not ok!!!")

test StaticTrie F:	ok
test DynamicTrie F:	ok


## ad 5. Porównanie czasów

In [17]:
dynamic_times={'A': {"no links":dtrieA.time, "links":dtrieA.time},
                'B': {"no links":dtrieB.time, "links":dtrieB.time},
                'C': {"no links":dtrieC.time, "links":dtrieC.time},
                'D': {"no links":dtrieD.time, "links":dtrieD.time},
                'E': {"no links":dtrieE.time, "links":dtrieE.time},
                'F': {"no links":dtrieF.time, "links":dtrieF.time}}

In [18]:
df_stimes=pd.DataFrame(dynamic_times)
display(df_stimes)

,A,B,C,D,E,F
no links,0.000014,0.000023,0.000019,0.0001,0.006528,8.376016
links,0.000014,0.000023,0.000019,0.0001,0.006528,8.376016
